<a href="https://colab.research.google.com/github/Hossein-Talebi1375/Lattice-Based-Structure/blob/main/Lattice_Based_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title #####**Central Hole Structure Simulation**

import matplotlib.pyplot as plt
import numpy as np

# Constants
NUM_NODES_X = 20  # Number of nodes in the X direction
NUM_NODES_Y = 20  # Number of nodes in the Y direction
SPRING_CONSTANT = 0.1  # Spring constant for regular springs
DAMPING_COEFFICIENT = 0.02  # Damping coefficient for regular springs
DISPLACEMENT_RATE = 0.1  # Constant rate of displacement for the top row nodes
CRITICAL_STRETCH = 1.5  # Critical stretch threshold for springs to break

# Generate node positions
nodes = np.zeros((NUM_NODES_Y, NUM_NODES_X, 2), dtype=float)
for i in range(NUM_NODES_Y):
    for j in range(NUM_NODES_X):
        nodes[i, j] = np.array([j, i])

# Initialize array to track broken springs
broken_springs = np.zeros((NUM_NODES_Y, NUM_NODES_X, 8), dtype=bool)

# Function to calculate the stretch of a spring
def calculate_stretch(node1, node2):
    return np.linalg.norm(node2 - node1) - 1  # Subtracting 1 for the rest length

# Function to update node positions based on springs
def update_positions(current_positions, time_step, broken_springs):
    updated_positions = np.copy(current_positions)

    # Define the radius of the hole
    hole_radius = 5

    for i in range(NUM_NODES_Y):
        for j in range(NUM_NODES_X):
            displacement = np.zeros(2)
            velocity = np.zeros(2)
            for k, (ni, nj) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]):
                if 0 <= i + ni < NUM_NODES_Y and 0 <= j + nj < NUM_NODES_X:
                    neighbor_position = current_positions[i + ni, j + nj]

                    # Define the center of the hole
                    hole_center = np.array([(NUM_NODES_X - 1) / 2, (NUM_NODES_Y - 1) / 2])

                    # Check if the node is within the hole
                    if np.linalg.norm(neighbor_position - hole_center) < hole_radius:
                        SPRING_CONSTANT_HOLE = 0  # Assign zero spring constant for nodes within the hole
                    else:
                        SPRING_CONSTANT_HOLE = SPRING_CONSTANT

                    delta = neighbor_position - current_positions[i, j]
                    distance = np.linalg.norm(delta)

                    # Check if the spring has exceeded the critical stretch
                    stretch = calculate_stretch(current_positions[i, j], neighbor_position)
                    if stretch > CRITICAL_STRETCH or broken_springs[i, j, k]:
                        force = np.zeros(2)
                        broken_springs[i, j, k] = True  # Mark the spring as broken
                    else:
                        # Calculate non-linear spring force with damping
                        force = SPRING_CONSTANT_HOLE * (distance - 1) * (delta / distance) + DAMPING_COEFFICIENT * (
                                (distance - 1) * (delta / distance) ** 3)
                        velocity = force * time_step
                        displacement += velocity * time_step

            updated_positions[i, j] += displacement

    # Add constant rate displacement to the top row nodes
    updated_positions[NUM_NODES_Y - 1, :, 1] += DISPLACEMENT_RATE * time_step
    updated_positions[NUM_NODES_Y - 1, :, 0] = np.arange(NUM_NODES_X)
    updated_positions[0, :, 1] = 0
    updated_positions[0, :, 0] = np.arange(NUM_NODES_X)

    return updated_positions, broken_springs

# Plot nodes and lines at initial positions
fig, ax = plt.subplots()

# Plot initial positions
for i in range(NUM_NODES_Y):
    for j in range(NUM_NODES_X):
        # Define the center of the hole
        hole_center = np.array([(NUM_NODES_X - 1) / 2, (NUM_NODES_Y - 1) / 2])
        hole_radius = 5
        # Check if the node is within the hole
        if np.linalg.norm(nodes[i, j] - hole_center) >= hole_radius:
            if i == NUM_NODES_Y - 1:
                # Draw arrows on the nodes at the top row
                plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')
                plt.arrow(nodes[i, j, 0], nodes[i, j, 1], 0, 1, head_width=0.4, head_length=0.6, fc='black', ec='black')
            elif i == 0:
                # Change the shape of nodes at the bottom row to red triangles
                plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'r^')
            else:
                plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')

# Plot connections at initial positions
for i in range(NUM_NODES_Y):
    for j in range(NUM_NODES_X):
        for ni, nj in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            if 0 <= i + ni < NUM_NODES_Y and 0 <= j + nj < NUM_NODES_X:
                neighbor_position = nodes[i + ni, j + nj]
                k = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)].index((ni, nj))
                stretch = calculate_stretch(nodes[i, j], neighbor_position)
                if np.linalg.norm(nodes[i, j] - hole_center) >= hole_radius:
                    if stretch > CRITICAL_STRETCH:
                        if not broken_springs[i, j, k]:
                            continue  # Skip plotting if the spring is not broken
                    else:
                        plt.plot([nodes[i, j, 0], neighbor_position[0]],
                                 [nodes[i, j, 1], neighbor_position[1]], 'gray')

plt.title("Initial Positions and Connections")
plt.pause(0.1)

# Update positions and plot every 500 steps
for step in range(1, 2501):
    # Clear the current axes
    ax.cla()
    plt.clf()

    # Update positions based on springs and displacement
    nodes, broken_springs = update_positions(nodes, time_step=1.0, broken_springs=broken_springs)

    # Plot nodes and lines every 500 steps
    if step % 500 == 0:

        for i in range(NUM_NODES_Y):
            for j in range(NUM_NODES_X):
                # Define the center of the hole
                hole_center = np.array([(NUM_NODES_X - 1) / 2, (NUM_NODES_Y - 1) / 2])
                hole_radius = 5
                # Check if the node is within the hole
                if np.linalg.norm(nodes[i, j] - hole_center) >= hole_radius:
                    if i == NUM_NODES_Y - 1:
                        # Draw arrows on the nodes at the top row
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')
                        plt.arrow(nodes[i, j, 0], nodes[i, j, 1], 0, 1, head_width=0.4, head_length=0.6, fc='black',
                                  ec='black')
                    elif i == 0:
                        # Change the shape of nodes at the bottom row to red triangles
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'r^')
                    else:
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')

        for i in range(NUM_NODES_Y):
            for j in range(NUM_NODES_X):
                for ni, nj in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
                    if 0 <= i + ni < NUM_NODES_Y and 0 <= j + nj < NUM_NODES_X:
                        neighbor_position = nodes[i + ni, j + nj]
                        k = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)].index((ni, nj))
                        stretch = calculate_stretch(nodes[i, j], neighbor_position)
                        if np.linalg.norm(nodes[i, j] - hole_center) >= hole_radius:
                            if stretch > CRITICAL_STRETCH:
                                if not broken_springs[i, j, k]:
                                    continue  # Skip plotting if the spring is not broken
                            else:
                                plt.plot([nodes[i, j, 0], neighbor_position[0]],
                                         [nodes[i, j, 1], neighbor_position[1]], 'gray')

        plt.title(f"Step: {step}")
        plt.pause(0.1)

plt.show()


In [ ]:
# @title #####**Edge Crack Structure Simulation**

import matplotlib.pyplot as plt
import numpy as np

# Constants
NUM_NODES_X = 21  # Number of nodes in the X direction
NUM_NODES_Y = 21  # Number of nodes in the Y direction
SPRING_CONSTANT = 0.1  # Spring constant for regular springs
DAMPING_COEFFICIENT = 0.02  # Damping coefficient for regular springs
DISPLACEMENT_RATE = 0.1  # Constant rate of displacement for the top row nodes
CRITICAL_STRETCH = 1.5  # Critical stretch threshold for springs to break

# Generate node positions
nodes = np.zeros((NUM_NODES_Y, NUM_NODES_X, 2), dtype=float)
for i in range(NUM_NODES_Y):
    for j in range(NUM_NODES_X):
        nodes[i, j] = np.array([j, i])

# Initialize array to track broken springs
broken_springs = np.zeros((NUM_NODES_Y, NUM_NODES_X, 8), dtype=bool)

# Function to calculate the stretch of a spring
def calculate_stretch(node1, node2):
    return np.linalg.norm(node2 - node1) - 1  # Subtracting 1 for the rest length

# Function to update node positions based on springs
def update_positions(current_positions, time_step, broken_springs):
    updated_positions = np.copy(current_positions)

    # Define the position of the crack
    crack_start=NUM_NODES_X // 2
    crack_end=NUM_NODES_X // 2

    for i in range(NUM_NODES_Y):
        for j in range(NUM_NODES_X):
            displacement = np.zeros(2)
            for k, (ni, nj) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]):
                if 0 <= i + ni < NUM_NODES_Y and 0 <= j + nj < NUM_NODES_X:
                    neighbor_position = current_positions[i + ni, j + nj]

                    # Check if the spring connects the two rows at the middle
                    if i == (crack_start and ( ni == crack_start or i + ni == crack_start + 1)) or (crack_start+1 and (i + ni == crack_start+1)) and j<=crack_end:
                        SPRING_CONSTANT_CRACK = 0  # Assign zero spring constant for the crack
                    else:
                        SPRING_CONSTANT_CRACK = SPRING_CONSTANT

                    delta = neighbor_position - current_positions[i, j]
                    distance = np.linalg.norm(delta)

                    # Check if the spring has exceeded the critical stretch
                    stretch = calculate_stretch(current_positions[i, j], neighbor_position)
                    if stretch > CRITICAL_STRETCH or broken_springs[i, j, k]:
                        force = np.zeros(2)
                        broken_springs[i, j, k] = True  # Mark the spring as broken
                    else:
                        force = SPRING_CONSTANT_CRACK * (distance - 1) * (delta / distance)
                        displacement += force

            updated_positions[i, j] += displacement

    # Add constant rate displacement to the top row nodes
    updated_positions[NUM_NODES_Y-1, :, 1] += DISPLACEMENT_RATE * time_step
    updated_positions[NUM_NODES_Y-1, :, 0] = np.arange(NUM_NODES_X)
    updated_positions[0, :, 1] = 0
    updated_positions[0, :, 0] = np.arange(NUM_NODES_X)

    return updated_positions, broken_springs

# Plot nodes and lines at initial positions
nodes = np.zeros((NUM_NODES_Y, NUM_NODES_X, 2), dtype=float)
for i in range(NUM_NODES_Y):
    for j in range(NUM_NODES_X):
        nodes[i, j] = np.array([j, i])

# Set zero spring constant value for springs connecting the two rows at the middle
crack_start = NUM_NODES_X // 2
crack_end = NUM_NODES_X // 2
crack_color = 'green'

# Plot nodes and connections
fig, ax = plt.subplots()

for i in range(NUM_NODES_Y):
    for j in range(NUM_NODES_X):
                    if i == NUM_NODES_Y - 1:
                        # Draw arrows on the nodes at the top row
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')
                        plt.arrow(nodes[i, j, 0], nodes[i, j, 1], 0, 1, head_width=0.4, head_length=0.6, fc='black',
                                  ec='black')
                    elif i == 0:
                        # Change the shape of nodes at the bottom row to red triangles
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'r^')
                    else:
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')

for i in range(NUM_NODES_Y):
    for j in range(NUM_NODES_X):
        for ni, nj in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            if 0 <= i + ni < NUM_NODES_Y and 0 <= j + nj < NUM_NODES_X:
                neighbor_position = nodes[i + ni, j + nj]
                if i == crack_start and j<=crack_end and ni==-1:
                    plt.plot([nodes[i, j, 0], neighbor_position[0]], [nodes[i, j, 1], neighbor_position[1]], crack_color)
                else:
                    plt.plot([nodes[i, j, 0], neighbor_position[0]], [nodes[i, j, 1], neighbor_position[1]], 'gray')

plt.title("Initial Positions and Connections with Left Edge Crack")
plt.pause(0.1)

# Update positions and plot every 10 steps
for step in range(1, 1501):
    # Clear the current axes
    ax.cla()
    plt.clf()

    # Update positions based on springs and displacement
    nodes, broken_springs = update_positions(nodes, time_step=1.0, broken_springs=broken_springs)

    # Plot nodes and lines every 10 steps
    if step % 250 == 0:
        for i in range(NUM_NODES_Y):
            for j in range(NUM_NODES_X):
                    if i == NUM_NODES_Y - 1:
                        # Draw arrows on the nodes at the top row
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')
                        plt.arrow(nodes[i, j, 0], nodes[i, j, 1], 0, 1, head_width=0.4, head_length=0.6, fc='black',
                                  ec='black')
                    elif i == 0:
                        # Change the shape of nodes at the bottom row to red triangles
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'r^')
                    else:
                        plt.plot(nodes[i, j, 0], nodes[i, j, 1], 'bo')

        for i in range(NUM_NODES_Y):
            for j in range(NUM_NODES_X):
                for ni, nj in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
                    if 0 <= i + ni < NUM_NODES_Y and 0 <= j + nj < NUM_NODES_X:
                        neighbor_position = nodes[i + ni, j + nj]
                        k = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)].index((ni, nj))
                        stretch = calculate_stretch(nodes[i, j], neighbor_position)
                        if stretch > CRITICAL_STRETCH or broken_springs[i, j, k]:
                            if broken_springs[i, j, k]:
                              continue
                        else:
                            plt.plot([nodes[i, j, 0], neighbor_position[0]], [nodes[i, j, 1], neighbor_position[1]], 'gray')

        plt.title(f"Step: {step}")
        plt.pause(0.1)

plt.show()
